In [ ]:
import pandas as pd
from scipy.stats import gamma, norm
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime


In [ ]:
def calculate_SPI(data):
    temp_data = pd.to_numeric(data, errors='coerce')
    temp_data = np.clip(temp_data, 1e-15, None)
    mask = np.isfinite(temp_data)
    temp_data = temp_data[mask]

    shape, loc, scale = gamma.fit(temp_data, floc=0)

    cdf = gamma.cdf(temp_data, shape, loc=0, scale=scale)
    spi = norm.ppf(cdf) 

    spi = (spi - np.mean(spi))/np.std(spi)
    return spi

In [ ]:
import os

l = []
def list_files(directory):
    files = []
    for file in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, file)):
            files.append(file)
    return files

l = list_files('./data')

In [ ]:
global_start_month = 1
global_start_year = 2015
global_end_month = 1
global_end_year = 2019


month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
             'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}

# SPI_df = pd.DataFrame(columns=["Station Name", "Latitude", "Longitude", "Avg SPI"])

def yearWiseCalculation(start_month,start_year,end_month,end_year, SPI_Type, make_plot=False):
    SPI_arr = []

    for files in l:
    # print(files)

        station_name, LatLong = files.strip('.csv').split('-')
        Lat, Long = LatLong.split(',')
        Lat = float(Lat.strip())
        Long = float(Long.strip())

        temp_df = pd.read_csv(os.path.join('data', files), header=1)
        dates = temp_df['Dates'].values
        # print(dates)
        # print(start_date, end_date)
        temp_df[['Month', 'Year']] = temp_df['Dates'].str.split('-', expand=True)
        temp_df['Year'] = temp_df['Year'].astype(int)
        temp_df['value'] = temp_df['ACTUAL (mm) ']
        temp_df['Month_num'] = temp_df['Month'].map(month_map)
        # print(temp_df.iloc[0]['Year'], temp_df.iloc[0]['Month'])
        # print(temp_df.iloc[0]['Year'], temp_df.iloc[0]['Month_num'])
        # print(temp_df.iloc[-1]['Year'], temp_df.iloc[-1]['Month_num'])
        if(temp_df.iloc[0]['Year']*12 + temp_df.iloc[0]['Month_num'] > start_year*12 + start_month):
            continue
        elif(temp_df.iloc[-1]['Year']*12 + temp_df.iloc[-1]['Month_num'] < end_year * 12 + end_month):
            continue

        temp_df['Date'] = pd.to_datetime(temp_df['Dates'], format='%b-%Y')

        # Set 'Date' column as the index
        temp_df.set_index('Date', inplace=True)

        start_date = datetime(global_start_year, global_start_month, 1)
        end_date = datetime(global_end_year, global_end_month, 1)

        # Create a complete date range spanning from the minimum to maximum date in the DataFrame
        complete_date_range = pd.date_range(start=start_date, end=end_date, freq='MS')

        # Reindex the DataFrame with the complete date range
        temp_df = temp_df.reindex(complete_date_range, fill_value=0)

        # Reset index to make 'Date' a column again
        temp_df.reset_index(inplace=True)

        # Rename 'index' column to 'Date'
        temp_df.rename(columns={'index': 'Date'}, inplace=True)
        # Convert 'Date' back to the original format (if needed)
        temp_df['Dates'] = temp_df['Date'].dt.strftime('%b-%Y')
        # print(temp_df['Dates'])
        

        temp_df[['Month', 'Year']] = temp_df['Dates'].str.split('-', expand=True)
        temp_df['Year'] = temp_df['Year'].astype(int)
        temp_df['value'] = temp_df['ACTUAL (mm) ']
        temp_df['Month_num'] = temp_df['Month'].map(month_map)
        
        temp_df = temp_df.drop(columns="ACTUAL (mm) ")

        rainfall_data = temp_df["value"].to_numpy()
        rainfall_final_data = temp_df["value"].to_numpy()
        rainfall_data = np.convolve(rainfall_data, np.ones(SPI_Type)/SPI_Type, mode='same')
        spi_values = calculate_SPI(rainfall_data)

        filtered_df_condition = ((temp_df['Year'] > start_year) | 
                            ((temp_df['Year'] == start_year) & (temp_df['Month_num'] >= start_month))) & ((temp_df['Year'] < end_year) |
                            ((temp_df['Year'] == end_year) & (temp_df['Month_num'] <= end_month)))

        first_idx = filtered_df_condition.idxmax()
        last_idx = filtered_df_condition[::-1].idxmax()     
        
        print(first_idx, last_idx)
        spi_filtered_values = spi_values[first_idx:last_idx+1]
        rainfall_final_filtered = rainfall_final_data[first_idx:last_idx+1]

        # average_spi = np.mean(spi_range)
        # filtered_df = temp_df[first_idx:last_idx+1]
        filtered_df = temp_df[first_idx:last_idx+1].copy()
        filtered_df['SPI'] = spi_filtered_values
        starting_date = start_date.strftime('%b-%Y')
        ending_date = end_date.strftime('%b-%Y')
        
        # FOR longest duration of drought
        avg_spi = np.mean(filtered_df['SPI'])
        mask = (spi_filtered_values < 0)
        longest_drought = 0
        current_drought = 0
        longest_drought_start = 0
        current_drought_start = 0
        for i, is_drought in enumerate(mask):
            if(is_drought):
                current_drought += 1
                if(current_drought == 1):
                    current_drought_start = i
                if(current_drought > longest_drought):
                    longest_drought = current_drought
                    longest_drought_start = current_drought_start
            else:
                current_drought = 0
        
        longest_drought_period = (longest_drought_start, longest_drought_start + longest_drought)
        print(f'Longest drought duration: {longest_drought} months')
        
        # print(starting_date, ending_date)
        # print(dates)
        # print(temp_df.iloc[0]['Year'], temp_df.iloc[0]['Month'])
        index1 = np.where(dates==starting_date)[0]
        index2 = np.where(dates==ending_date)[0]
        # many dates arrays dont have the starting date and ending date entered by the user
        if(index1.size == 0 or index2.size == 0):
            # continue
            # print(dates)
            # Convert date strings to datetime objects
            date_objs = [datetime.strptime(date_str, '%b-%Y') for date_str in dates]

            # Convert starting_date and ending_date strings to datetime objects
            starting_date_obj = datetime.strptime(starting_date, '%b-%Y')
            ending_date_obj = datetime.strptime(ending_date, '%b-%Y')

            # Find the indices of the first date after the starting date
            index1 = next((i for i, date_obj in enumerate(date_objs) if date_obj > starting_date_obj), None)

            # Find the indices of the last date before the ending date
            index2 = next((i for i, date_obj in reversed(list(enumerate(date_objs))) if date_obj < ending_date_obj), None)

            # find first date in dates after starting date
            # index1 = np.where(dates > starting_date)[0]
            # find first date in dates before ending date
            # index2 = np.where(dates < ending_date)[0]
            # SPI_arr.append([station_name, Lat, Long, np.average(filtered_df['SPI']), 0])
            # continue
        else:
            index1 = index1[0]
            index2 = index2[0]

        # print(index1[0], index2[0])
        index1 =index1 - SPI_Type + 1
        index2 = index2 - SPI_Type + 1
        # print(index1, index2)
        spi_range = spi_values[index1:index2+1]

        drought_frequency = 0
        consecutive_negatives = 0

        drought_frequency = np.sum(spi_range < 0)

        if(make_plot):
            plt.figure(figsize=(20,12))
            plt.title(station_name)
            plt.plot(filtered_df['Dates'], filtered_df['SPI'])
            plt.xticks(filtered_df['Dates'], rotation=90)
            plt.ylim(-max(abs(filtered_df['SPI'])+2), max(abs(filtered_df['SPI'])+2))
            plt.fill_between(filtered_df['Dates'], filtered_df['SPI'], where=(filtered_df['SPI'] > 0), color='blue', alpha=0.5, label='Positive SPI')
            plt.fill_between(filtered_df['Dates'], filtered_df['SPI'], where=(filtered_df['SPI'] <= 0), color='red', alpha=0.5, label='Negative SPI')
            if longest_drought > 0:
                plt.fill_between(filtered_df['Dates'][longest_drought_period[0]:longest_drought_period[1]], filtered_df['SPI'][longest_drought_period[0]:longest_drought_period[1]], where=(filtered_df['SPI'][longest_drought_period[0]:longest_drought_period[1]] <= 0), color='green', alpha=0.5, label='Longest Drought Period')
            # legend 
            plt.legend()
            plt.show()

        # SPI_df.iloc[len(SPI_df.index)] = [station_name, Lat, Long, np.average(filtered_df['SPI'])]
        SPI_arr.append([station_name, Lat, Long, np.average(filtered_df['SPI']), drought_frequency, np.mean(rainfall_final_filtered)])
    return SPI_arr



In [ ]:
from scipy.interpolate import griddata
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin, Affine


def interpolate_grid(SPI_df):
    area_of_interest = gpd.read_file("final-basin.gpkg")


    xmin, ymin, xmax, ymax = area_of_interest.total_bounds
    # print(xmin, ymin, xmax, ymax)
    grid_x, grid_y = np.meshgrid(np.linspace(xmin, xmax, num=100), np.linspace(ymin, ymax, num=100))

    grid_z = griddata((SPI_df['Longitude'], SPI_df['Latitude']), SPI_df['Avg SPI'], (grid_x, grid_y), method='cubic')
    spi_value_layer=grid_z
    
    interpolated_grid = gpd.GeoDataFrame(
        {'value': grid_z.flatten()},
        geometry=gpd.points_from_xy(grid_x.flatten(), grid_y.flatten()), crs=area_of_interest.crs
    )
    clipped_interpolated_grid = gpd.clip(interpolated_grid, area_of_interest)

    # plt.figure(figsize=(12, 8))
    area_of_interest.plot(figsize=(12, 8), color='white', edgecolor='black')
    clipped_interpolated_grid.plot(column='value',  ax=plt.gca(), legend=True)
    plt.scatter(SPI_df['Longitude'], SPI_df['Latitude'], c=SPI_df['Avg SPI'], edgecolor='black', label='Data Points')

    for i, txt in enumerate(SPI_df['Station Name']):
        plt.annotate(txt, (SPI_df['Longitude'].iloc[i], SPI_df['Latitude'].iloc[i]), fontsize=8, ha='left')

    plt.show()
    
    final_df = pd.DataFrame({
        'SPI': spi_value_layer.flatten(),
    })

    # final_df = final_df.dropna(subset=['SPI'])
    # final_df = final_df.reset_index(drop=True)

    output_file_name = 'SPI_raster.tif'

    number_of_rows = spi_value_layer.shape[0]
    number_of_columns = spi_value_layer.shape[1]

    left, right = grid_x.min(), grid_x.max()
    bottom, top = grid_y.min(), grid_y.max()

    # print(grid_y)
    resolution_x = 0.032
    resolution_y = 0.026

    # print(grid_x)
    transform = from_origin(left, bottom, resolution_x, -resolution_y) # negative sign in y to flip in the y direction

    # transform = Affine(resolution, 0, left, 0, -resolution, top)

    options = {
        'driver': 'GTiff',
        'dtype': final_df['SPI'].dtype,
        'width': number_of_columns,
        'height': number_of_rows,
        'count': 1,
        'crs': 'EPSG:4326',
        'transform': transform,
        'nodata': np.nan
    }

    with rasterio.open(output_file_name, 'w', **options) as output_file:
        output_file.write(spi_value_layer, 1)



In [ ]:
def interpolate_avg_rainfall(SPI_df):
    area_of_interest = gpd.read_file("final-basin.gpkg")


    xmin, ymin, xmax, ymax = area_of_interest.total_bounds
    # print(xmin, ymin, xmax, ymax)
    grid_x, grid_y = np.meshgrid(np.linspace(xmin, xmax, num=100), np.linspace(ymin, ymax, num=100))

    grid_z = griddata((SPI_df['Longitude'], SPI_df['Latitude']), SPI_df['Rainfall'], (grid_x, grid_y), method='cubic')
    spi_value_layer=grid_z
    
    interpolated_grid = gpd.GeoDataFrame(
        {'value': grid_z.flatten()},
        geometry=gpd.points_from_xy(grid_x.flatten(), grid_y.flatten()), crs=area_of_interest.crs
    )
    clipped_interpolated_grid = gpd.clip(interpolated_grid, area_of_interest)

    # plt.figure(figsize=(12, 8))
    area_of_interest.plot(figsize=(12, 8), color='white', edgecolor='black')
    clipped_interpolated_grid.plot(column='value',  ax=plt.gca(), legend=True)
    plt.scatter(SPI_df['Longitude'], SPI_df['Latitude'], c=SPI_df['Rainfall'], edgecolor='black', label='Data Points')

    for i, txt in enumerate(SPI_df['Station Name']):
        plt.annotate(txt, (SPI_df['Longitude'].iloc[i], SPI_df['Latitude'].iloc[i]), fontsize=8, ha='left')

    plt.show()

In [ ]:
from scipy.interpolate import griddata
import geopandas as gpd

def drought_frequency(SPI_df):
    area_of_interest = gpd.read_file("final-basin.gpkg")

    xmin, ymin, xmax, ymax = area_of_interest.total_bounds
    # print(xmin, ymin, xmax, ymax)
    grid_x, grid_y = np.meshgrid(np.linspace(xmin, xmax, num=100), np.linspace(ymin, ymax, num=100))

    grid_z = griddata((SPI_df['Longitude'], SPI_df['Latitude']), SPI_df['Drought Frequency'], (grid_x, grid_y), method='nearest')
    interpolated_grid = gpd.GeoDataFrame(
        {'value': grid_z.flatten()},
        geometry=gpd.points_from_xy(grid_x.flatten(), grid_y.flatten()), crs=area_of_interest.crs
    )
    clipped_interpolated_grid = gpd.clip(interpolated_grid, area_of_interest)

    # plt.figure(figsize=(12, 8))
    area_of_interest.plot(figsize=(12, 8), color='white', edgecolor='black')
    clipped_interpolated_grid.plot(column='value',  ax=plt.gca(), legend=True)
    plt.scatter(SPI_df['Longitude'], SPI_df['Latitude'], c=SPI_df['Drought Frequency'], edgecolor='black', label='Data Points')

    for i, txt in enumerate(SPI_df['Station Name']):
        plt.annotate(txt, (SPI_df['Longitude'].iloc[i], SPI_df['Latitude'].iloc[i]), fontsize=8, ha='left')

    plt.show()

In [ ]:
from scipy.interpolate import griddata
import geopandas as gpd
import numpy as np

def areaOfExtent(SPI_df):
    area_of_interest = gpd.read_file("final-basin.gpkg")

    xmin, ymin, xmax, ymax = area_of_interest.total_bounds
    grid_x, grid_y = np.meshgrid(np.linspace(xmin, xmax, num=100), np.linspace(ymin, ymax, num=100))

    # Calculate drought frequency
    grid_z_frequency = griddata((SPI_df['Longitude'], SPI_df['Latitude']), SPI_df['Avg SPI'], (grid_x, grid_y), method='cubic')
    interpolated_grid_frequency = gpd.GeoDataFrame(
        {'frequency': grid_z_frequency.flatten()},
        geometry=gpd.points_from_xy(grid_x.flatten(), grid_y.flatten()), crs=area_of_interest.crs
    )
    clipped_interpolated_grid_frequency = gpd.clip(interpolated_grid_frequency, area_of_interest)


    # print(grid_z_frequency)
    # # Calculate drought areal extent
    
    
    count=0
    drought=0

    for i in grid_z_frequency:
        for j in i:
            # print(j)
            if(np.isnan(j)!=True):
                count+=1
                if(j<0):
                    drought+=1                

    
    return drought/count


In [ ]:
start_month = int(input("Enter start month: "))
start_year = int(input("Enter start year: "))
end_month = int(input("Enter end month: "))
end_year = int(input("Enter End Year: "))
SPI_Type = int(input("Enter SPI Value: "))

user_start_year = start_year

# %%
print(start_month, start_year, end_month, end_year)

SPI_arr = yearWiseCalculation(start_month,start_year,end_month,end_year, SPI_Type, make_plot=True)

SPI_df = pd.DataFrame(SPI_arr, columns=["Station Name", "Latitude", "Longitude", "Avg SPI", "Drought Frequency", "Rainfall"])


In [ ]:
areaOfExtent(SPI_df)

In [ ]:
SPI_df

# Average Rainfall in the selected timeframe

In [ ]:
interpolate_avg_rainfall(SPI_df)

# Drought frequency interpolation

In [ ]:
drought_frequency(SPI_df)

# Spatial interpolation of SPI Values  (Intensity)

In [ ]:
interpolate_grid(SPI_df)

# Areal Extent for the current timeframe

In [ ]:
print('Arial extent (No of cells with SPI<0 / total no of cells): ', areaOfExtent(SPI_df))

# Arial Extent per year in the selected timeframe

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have calculated areal extent values and stored them in a list called 'areal_extent_values' for each year

start_month = 1
end_month = 12
years = range(user_start_year,end_year+1)

areal_extent_values = []

# Loop through each year and calculate areal extent
for start_year in years:
    SPI_arr = yearWiseCalculation(start_month, start_year, end_month, start_year, SPI_Type)
    SPI_df = pd.DataFrame(SPI_arr, columns=["Station Name", "Latitude", "Longitude", "Avg SPI", "Drought Frequency", "Rainfall"])
    areal_extent = areaOfExtent(SPI_df)
    if(start_year == 2019):
        print(areal_extent)
    areal_extent_values.append(areal_extent)

# Plotting the bar graph

print(areal_extent_values)
plt.figure(figsize=(10, 6))
plt.bar(years, areal_extent_values, color='blue')
plt.title("Areal Extent of Drought Over Years")
plt.xlabel("Year")
plt.ylabel("Areal Extent")
plt.xticks(years)  # Ensure all years are displayed on the x-axis
plt.grid(axis='y')  # Add gridlines for better readability
plt.show()
